In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import tensorflow as tf
from sklearn.model_selection import cross_val_score, train_test_split, KFold

def load_data(nao_index_train_path = './nao_index_train.npy',
              tas_train_path = './tas_train.npy',
              psl_train_path = './psl_train.npy'):

    x1 = np.load(tas_train_path)
    x2 = np.load(psl_train_path)
    y = np.load(nao_index_train_path)
    return x1, x2, y

def cross_run():
    for train_index, test_index in kf.split(xnpy, ynpy):
        kf = KFold(n_splits=10)
        data = kf.get_n_splits(np.concatenate((xnpy,ynpy)))
    
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_val = data[train_index], data[test_index]
        y_train, y_val = y[train_index], y[test_index]
    
        # Convert to Tensors
        # Training set
        X_train_tensor = tf.convert_to_tensor(X_train, dtype=np.float32, dtype_hint=None, name=None)
        y_train_tensor = tf.convert_to_tensor(y_train, dtype=np.float32, dtype_hint=None, name=None)
    
        # def tensor_convert(x1 = temp_train, x2 = pres_train):
        #     y1 = tf.convert_to_tensor(x1, dtype=np.float32, dtype_hint=None, name=None)
        #     y2 = tf.convert_to_tensor(x2, dtype=np.float32, dtype_hint=None, name=None)
        #     return y1, y2
        # temp_tensor, pres_tensor = tensor_convert()

        # Validation set
        X_val_tensor = tf.convert_to_tensor(X_val, dtype=np.float32, dtype_hint=None, name=None)
        y_val_tensor = tf.convert_to_tensor(y_val, dtype=np.float32, dtype_hint=None, name=None)
    
        # Data loader
        train_ds = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))
        val_ds = tf.data.Dataset.from_tensor_slices((X_val_tensor, y_val_tensor))
    
        train_ds = train_ds.shuffle(buffer_size=50, seed=42).batch(64)
    

In [6]:


temp_train, pres_train, nao_index_train = load_data()

In [7]:
np.shape(temp_train), np.shape(pres_train), np.shape(nao_index_train)

((900, 2322), (900, 2322), (900,))

In [8]:
# # PCA with 50 components
# pca = PCA(n_components=50)
# pca.fit(temp_train)
# temp_train = pca.transform(temp_train)
# pca.fit(pres_train)
# pres_train = pca.transform(pres_train)

In [9]:
# Reshape the data as per lat & lon
temp_train = np.reshape(temp_train,(900,43,54,1))
pres_train = np.reshape(pres_train,(900,43,54,1))

In [10]:
X_comb = np.concatenate((temp_train,pres_train), axis =3)
y = nao_index_train
np.shape(X_comb), np.shape(y)

((900, 43, 54, 2), (900,))

In [11]:
# # Random split CV
# X_train, X_val, y_train, y_val= train_test_split(X_comb, y, test_size=0.1, random_state=42)
# np.shape(X_train), np.shape(X_val), np.shape(y_train), np.shape(y_val)

In [12]:
# KFolding CV to k=10 splits
kf = KFold(n_splits=10)
kf.get_n_splits(X_comb)
print(kf)

KFold(n_splits=10, random_state=None, shuffle=False)


In [17]:
for train_index, test_index in kf.split(X_comb):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_val = X_comb[train_index], X_comb[test_index]
    y_train, y_val = y[train_index], y[test_index]
    
    # Convert to Tensors
    # Training set
    X_train_tensor = tf.convert_to_tensor(X_train, dtype=np.float32, dtype_hint=None, name=None)
    y_train_tensor = tf.convert_to_tensor(y_train, dtype=np.float32, dtype_hint=None, name=None)
    
    # def tensor_convert(x1 = temp_train, x2 = pres_train):
    #     y1 = tf.convert_to_tensor(x1, dtype=np.float32, dtype_hint=None, name=None)
    #     y2 = tf.convert_to_tensor(x2, dtype=np.float32, dtype_hint=None, name=None)
    #     return y1, y2
    # temp_tensor, pres_tensor = tensor_convert()

    # Validation set
    X_val_tensor = tf.convert_to_tensor(X_val, dtype=np.float32, dtype_hint=None, name=None)
    y_val_tensor = tf.convert_to_tensor(y_val, dtype=np.float32, dtype_hint=None, name=None)
    
    # Data loader
    train_ds = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))
    val_ds = tf.data.Dataset.from_tensor_slices((X_val_tensor, y_val_tensor))
    
    train_ds = train_ds.shuffle(buffer_size=50, seed=42).batch(64)
    
    ---> MODELfit
    ---> EVal